<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png"/>

<img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>   

# Tokenización con transformers transformers

En este notebook, aprenderemos a preprocesar los textos para poder ser usados por los modelos transformadores.

Si planeas usar un determinado modelo pre-entrenado (por ejemplo, BERT, GPT o XLNet), siempre deberás utlizar el tokenizador que fue entrenado con el modelo. Es importante porque el tokenizador utilizará el vocabulario creado durante el pre-entrenamiento del transforme, y además dividirá los textos siguiendo el mismo método utilizado para los textos del corpus utilizado para el pre-entrenamiento del modelo transformer. 


Puedes encontrar más información en el siguiente [link](#https://huggingface.co/docs/transformers/v4.14.1/en/preprocessing)

## Tokenizador

Un tokenizador divide un texto en tokens (palabras o partes de palabras, signos de puntuación, etc.).

Además, el tokenizador va a traducir cada token a un número, su índice en el vocabulario del modelo.De esta forma, cada texto tokenizador se va a representar como un vector (tensor) de números enteros.

El primer paso será instalar la librería **transformers**

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 7.9 MB/s eta 0:00:00



La librería **transformers** proporciona una clase **AutoTokenizer** que nos permitirá cargar facilmente el tokenizador de cualquier modelo transformer. En nuestro caso, vamos a cargar el tokenizador asociado a uno de los modelos transfomers más utilizadores, [**bert-base-cased**](#https://huggingface.co/bert-base-cased).

**cased**: indica que el modelo distingue entre mayúsculas y minúsculas. 


In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')


Vamos a comenzar tokenizando una sencilla oración. 

In [4]:
encoded_input = tokenizer("Hello, I'm a single sentence!")
print(encoded_input)

{'input_ids': [101, 8667, 117, 146, 112, 182, 170, 1423, 5650, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Podemos ve que el tokenizador devuelve un diccionario con los siguientes campos:

- **input_ids** que como valor asociado tiene una lista de números enteros. En concreto, son los índices de cada token en nuestra oración. 
- **token_type_ids**, su valor asociado es una lista de 0's y 1's. 0 indica que el token pertenece a una primera oración, y 1 indica que pertenece a la segunda oración. En nuestro caso, como únicamente se ha procesado una oración, la lista únicamente contiene 0's. Veremos un ejemplo con dos oraciones en las siguientes celdas. 

- **attention_mask**, su valor asociado también sería una lista de 0's y 1's. 0 representa a un token de padding, y 1 un token real de la oración. En nuestro ejemplo, como no hemos aplicado ningún padding, todos los elementos de la lista son 1. 


En las siguientes celdas, volveremos a estudiar con más detalles estas dos listas **token_type_ids** y **attention_mask**. Por el momento, nos centraremos únicamente en *token_type_ids*.



El tokenizador también nos permite decodificar una lista de enteros a sus tokens correspondientes. 

En la siguiente celda, vamos a decodificar la lista **input_ids** que hemos obtenido anteriormente. Para ello, usaremos el método **decode**. 


Fijate que el tokenizador ha añadido automáticamente dos tokens especiales a la oración: 
- [𝐶𝐿𝑆] indica el comienzo de la primera oración, y 
- [𝑆𝐸𝑃] indica el final de cada oración.


In [5]:
tokenizer.decode(encoded_input["input_ids"])

"[CLS] Hello, I'm a single sentence! [SEP]"

La salida de los tokenizadores no es igual para todos los modelos. 

Por ejemplo, el tokenizador asociado al modelo **gpt2-medium**, cuando procesa una oración, devueve una lista de **input_ids** y también **attention_mask**, pero no devuelve **token_type_ids**. 

¿Por qué esta diferencia?. Esto es porque BERT es entrenado utilizando dos estrategias: 
1) predecir tokens enmascarados, y 
2) predecir si dos oraciones son contiguas (por eso necesita distinguir entre los tokens de la primera oración y los de la segunda (**token_type_ids**).
El modelo GPT no utiliza la segunda estrategia, y por tanto, no va a necesitar distinguir si un token es de una oración o de la siguiente. 

Veamos la salida producida por el tokenizador de **gpt2-medium** para la misma oración.

Además, este tokenizador no añade tampoco los tokens especiales [𝐶𝐿𝑆] y [SEP], que si eran añadidos por el tokenizador de BERT.

In [7]:
tokenizer_gpt2 = AutoTokenizer.from_pretrained('gpt2-medium')
encoded_input = tokenizer_gpt2("Hello, I'm a single sentence!")
print(encoded_input)
print(tokenizer_gpt2.decode(encoded_input["input_ids"]))

{'input_ids': [15496, 11, 314, 1101, 257, 2060, 6827, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
Hello, I'm a single sentence!


Volvamos al tokenizador de BERT. Vamos usarlo para tokenizar una lista de oraciones: 


In [13]:
sentences = ["Hello I'm a single sentence",
                   "This is the second sentence",
                   "And this is the very very very very last one"]
encoded_inputs = tokenizer(sentences)
print(encoded_inputs)



{'input_ids': [[101, 8667, 146, 112, 182, 170, 1423, 5650, 102], [101, 1188, 1110, 1103, 1248, 5650, 102], [101, 1262, 1142, 1110, 1103, 1304, 1304, 1304, 1304, 1314, 1141, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


Ahora en cada uno de los campos **input_ids**, **token_type_ids**, o **attention_mask**, tiene asociado una lista de listas. Cada sublista se corresponde a una oración. 

Vamos a fijarnos únicamente en los **input_ids**. En todos se ha asociado los token especiales, para indicar el comienzo y final de cada oración.
Además, podemos ver que cada lista tiene un número distinto de tokens.  

In [14]:
print(encoded_inputs.input_ids[0])
print(encoded_inputs.input_ids[1])
print(encoded_inputs.input_ids[2])

[101, 8667, 146, 112, 182, 170, 1423, 5650, 102]
[101, 1188, 1110, 1103, 1248, 5650, 102]
[101, 1262, 1142, 1110, 1103, 1304, 1304, 1304, 1304, 1314, 1141, 102]


## Padding y Truncation

Cuando pasamos un lote (batch) de oraciones al modelo, vamos a necesitar que todas las oraciones del mismo lote tengan la misma longitud. 

Por tanto, necesitamos que el tokenizador sea capaz de:
- aplicar padding, es decir, completar una oración (con un token especial PAD) para que tenga el mismo número de tokens que la oración más larga en el mismo lote, o a un número máximo de tokens.  
- aplicar truncation para recortar oraciones. Los modelos transformer suelen tener un límite de 512 tokens para la entrada, por tanto, se truncarán todas las oracioens que tengan una longitud mayor que esa. También se puede considera una longtiud máxima para las oraciones, y truncar en esa longitud.
- además, necesitamos devolver las listas en formato tensor, para que puedan ser procesadas por el modelo. 


El tokenizador puede cumplir los tres objetivos, gracias a la definición de los argumentos: **padding**, **truncation**, y **return_tensors**, como se ve en el siguiente ejemplo:


In [17]:
batch = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
batch['input_ids']

tensor([[ 101, 8667,  146,  112,  182,  170, 1423, 5650,  102,    0,    0,    0],
        [ 101, 1188, 1110, 1103, 1248, 5650,  102,    0,    0,    0,    0,    0],
        [ 101, 1262, 1142, 1110, 1103, 1304, 1304, 1304, 1304, 1314, 1141,  102]])

Puedes ver que todas las oraciones  tienen la misma longitud (que es la longitud de la oración más larga). 
En las dos primeras oraciones, podemos ver como se han añadido tokens de padding (0). 

Vamos a fijarnos ahora en el campo **attention_mask**:

In [18]:
batch['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

En la tercera oración, ¿por qué todos los elementos son 1?. El motivo es porque todos sus tokens son tokens reales, es decir, no hay tokens de padding. 
Sin embargo, en las dos primeras oraciones, sí podemos ver que hay 0's, que coinciden con los tokens de padding. 


A veces nuestros textos están lejos de tener 512 tokens. Si utilizamos entradas de 512 tokens, estaríamos desperdiciando mucha memoria. En ese caso, nos va a interesar definir una longitud máxima. 

En el siguiente ejemplo, vamos a limitar la longitud máxima a 8 tokens. 


In [22]:
batch = tokenizer(sentences, padding=True, truncation=True, max_length=8, return_tensors="pt")
batch['input_ids']

tensor([[ 101, 8667,  146,  112,  182,  170, 1423,  102],
        [ 101, 1188, 1110, 1103, 1248, 5650,  102,    0],
        [ 101, 1262, 1142, 1110, 1103, 1304, 1304,  102]])

¿Qué pasa si descodificamos los **input_ids** de la tercera oración?. Podemos ver que se han perdido los últimos tokens de la oración. 

In [24]:
print(batch.input_ids[2])
print(sentences[2])
tokenizer.decode(batch.input_ids[2])

tensor([ 101, 1262, 1142, 1110, 1103, 1304, 1304,  102])
And this is the very very very very last one


'[CLS] And this is the very very [SEP]'


Veamos con más detalle cada uno de estos argumentos:

- **max_length**: controla la longitud para las operaciones de padding y truncation. Si no se usa este parámetro, su valor será la longitud máxima que el modelo puede aceptar (normalmente 512 tokens). 
- **padding**: su valor puede ser:
    - True o 'longest' añaden padding considerando la oración más larga en el batch.
    - 'max_length' añade padding considerando la longitud máxima indicada en el argumento **max_length**.
    - False o 'do_not_pad', no se aplica padding. 
    
- **Truncation**, su valor puede ser:
    - True o 'longest_first' trunca las oraciones a la longitud máxima que se ha especificado **max_length** o la máxima longitud permitida por el modelo. Si hay dos oraciones, se trunca token por token en la oración más larga del par, hasta alcanzar la longitud adecuada. 
    
- 'only_second' trunca las oraciones a la longitud máxima que se ha especificado **max_length** o la máxima longitud permitida por el modelo. Solo truncará la segunda oración de un par si se proporciona un par de secuencias (o un lote de pares de secuencias).

- 'only_first'  trunca las oraciones a la longitud máxima que se ha especificado **max_length** o la máxima longitud permitida por el modelo. Solo truncará la primera oración de un par si se proporciona un par de secuencias (o un lote de pares de secuencias).

- False 0 'do_not_truncate' no trunca la oración. 
